In [2]:
import requests
import math

APP_ID = "abc4f19e"
APP_KEY = "6d65fe87e0bef00c8ed97b191f2bad3a"

results_per_page = 50
max_records = 6000
records = []

page = 1

while len(records) < max_records:
    url = f"https://api.adzuna.com/v1/api/jobs/gb/search/{page}"
    params = {
        "app_id": APP_ID,
        "app_key": APP_KEY,
        "results_per_page": results_per_page,
        "what": "developer",
        "content-type": "application/json"
    }

    response = requests.get(url, params=params)
    data = response.json()

    records.extend(data["results"])

    if len(records) >= data["count"]:
        break

    page += 1

print("Total records collected:", len(records))

Total records collected: 6000


In [3]:
import json
import re

def clean_html(text):
    return re.sub(r"<.*?>", "", text)

job_descriptions = [
    {
        "id": job.get("id"),
        "title": job.get("title"),
        "company": job.get("company", {}).get("display_name"),
        "description": clean_html(job.get("description", "")).strip()
    }
    for job in records
    if job.get("description")
]

print("Total job descriptions:", len(job_descriptions))


Total job descriptions: 6000


In [5]:
with open("adzuna_job_descriptions_6000.json", "w", encoding="utf-8") as f:
    json.dump(job_descriptions, f, ensure_ascii=False, indent=2)

print("JSON file saved")


JSON file saved


In [6]:
from google.colab import files
files.download("adzuna_job_descriptions_6000.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>